# <h1 style="text-align: center;">Data Managements - HW3 - Group 21</h1>


| STUDENT | ID |
| :-: | :-: |
| Francesco Lazzari | 1917922 |  
|  Riccardo Violano | 2148833 |    

### Load Packages

In [2]:
import pandas as pd
import numpy as np

from cassandra.cluster import Cluster
from cassandra.cluster import Session

import ipywidgets as widgets  
from IPython.display import display, clear_output  
from datetime import datetime  

----

## <h2 style="text-align: center;">Preprocessing</h2>

----

Here we made some preprocessing on the table of the previous assignment in order to fix some datatype.

In [8]:
# Import the .csv files with the records of the HW2 tables obtained throught the previous preprocessing script

airlines = pd.read_csv('/Users/francescolazzari/Documents/Università - Data Science/DM/Homework/HW3/Data/airlines.csv')
airports = pd.read_csv('/Users/francescolazzari/Documents/Università - Data Science/DM/Homework/HW3/Data/airports_refined.csv')
plane_model = pd.read_csv('/Users/francescolazzari/Documents/Università - Data Science/DM/Homework/HW3/Data/plane_model.csv')
planes = pd.read_csv('/Users/francescolazzari/Documents/Università - Data Science/DM/Homework/HW3/Data/planes_refined.csv')
weathers = pd.read_csv('/Users/francescolazzari/Documents/Università - Data Science/DM/Homework/HW3/Data/weathers_refined.csv')
flights = pd.read_csv('/Users/francescolazzari/Documents/Università - Data Science/DM/Homework/HW3/Data/flights_refined.csv')

In [9]:
# Datatype correction for the table 'PLANES'

# three columns had the datatype 'float' instead of 'int'

planes['year'] = pd.to_numeric(planes['year'], errors='coerce')  
planes['year'] = planes['year'].astype('Int64')

planes['engines'] = pd.to_numeric(planes['engines'], errors='coerce')  
planes['engines'] = planes['engines'].astype('Int64')

planes['seats'] = pd.to_numeric(planes['seats'], errors='coerce')  
planes['seats'] = planes['seats'].astype('Int64')

planes['speed'] = pd.to_numeric(planes['speed'], errors='coerce')  
planes['speed'] = planes['speed'].astype('Int64')

In [10]:
# Datatype correction for the table 'PLANES'

# the 'time_hour' column had the 'str' datatype instead 'datetime'
weathers['time_hour']= pd.to_datetime(weathers['time_hour'])

# the 'wind_dir' column had the 'float' datatype intead 'int'
weathers['wind_dir'] = pd.to_numeric(weathers['wind_dir'], errors='coerce')  
weathers['wind_dir'] = weathers['wind_dir'].astype('Int64')

In [11]:
# Datatype correction for the table 'PLANES'

# the 'sched_dep_time' and 'sched_arr_time' columns had the 'str' datatype instead 'datetime'
flights['sched_dep_time']= pd.to_datetime(flights['sched_dep_time'])
flights['sched_arr_time']= pd.to_datetime(flights['sched_arr_time'])

# three columns had the datatype 'float' instead of 'int'

flights['dep_delay_min'] = pd.to_numeric(flights['dep_delay_min'], errors='coerce')  
flights['dep_delay_min'] = flights['arr_delay_min'].astype('Int64')

flights['arr_delay_min'] = pd.to_numeric(flights['arr_delay_min'], errors='coerce') 
flights['arr_delay_min'] = flights['arr_delay_min'].astype('Int64')

flights['air_time_min'] = pd.to_numeric(flights['air_time_min'], errors='coerce')  
flights['air_time_min'] = flights['air_time_min'].astype('Int64')

----

## <h2 style="text-align: center;">Cassandra Setup</h2>

----

### Installation

1. Install Docker Desktop.  

   https://www.docker.com/products/docker-desktop/

2.  Open the terminal.

3.  Create and start a Cassandra container exposing port 9042:

   `docker run --name my-cassandra -d -p 9042:9042 cassandra:latest`

4.  Edit the `cassandra.yaml` file to enable Materialized Views (experimental feature).

   - Open the `my-cassandra` container in Docker.

   - Go to the files section.

   - Navigate to the path `/etc/cassandra`.

   - Open the `cassandra.yaml` file with a text editor (there's one included in Docker).

   - Modify the following line to enable Materialized Views:
   
      `enable_materialized_views: true`

5.  Restart the container to apply the changes:
   
    `docker restart my-cassandra`

6.  Verify the status of port 9042 (optional):
   
    `netstat -an | grep 9042`

   We expect an output such this

    `tcp46      0      0  *.9042                 *.*                    LISTEN `
  

To stop the container at the end of your work, use the following code:  
   
`docker stop my-cassandra`

To start the container again, use the following code instead of the one used in step (3):
   
`docker start my-cassandra`

### Remote connection to Cassandra 

Since Cassandra doesn't have a visual interface and can be used only from the terminal we chose to interact with it remotely with the `cassandra.cluster` python package.

First we have to create a connection with our cluster and since cassandra is running locally in **Docker**, the `localhost` parameter substitute the usual ***host name*** and ***port*** and ensure the connection to our ***Docker container***.

In [3]:
cluster = Cluster(['localhost'])
session = cluster.connect()

### Keyspace cration

To work with Cassandra we need to create and connect to a keyspace which is (more or less) the equivalent of a schema in a RDB, and this is the Syntax to do that.

In [7]:
session.execute("""DROP KEYSPACE IF EXISTS dm_hw3;""") 

In [8]:
keyspace_query = '''
CREATE KEYSPACE IF NOT EXISTS DM_HW3
WITH REPLICATION = {'class': 'SimpleStrategy', 'replication_factor': 1};
'''

session.execute(keyspace_query)

Here we check if the keyspace was created correctly.

In [4]:
rows = session.execute("SELECT keyspace_name FROM system_schema.keyspaces")

# print the keyspace to check
for row in rows:
    print(row)

Row(keyspace_name='system_auth')
Row(keyspace_name='dm_hw3')
Row(keyspace_name='system_schema')
Row(keyspace_name='system_distributed')
Row(keyspace_name='system')
Row(keyspace_name='system_traces')


In [5]:
session.set_keyspace('dm_hw3')

----

## <h2 style="text-align: center;">Table creation</h2>

----

Since we are using a `NoSQL` database which doesn't allow join/relationships between tables we nedded to redesign our previous RDB. This peculiar feature of nonrelational databases implies the need to denormalize tables by often reporting the same information on different tables (Relaxed consistency). 

For this fact to re-do the queries that we did in the Homework 1-2 we needed also the help of some pandas tool in python to compensate some lacks of Cassandra, an example will be how cassandra works with the operator "max", when we use the operator Max on a feature Cassandra doesen't register all the information about that maximum so the results can be a bit weird with a mixed-up result, we showed to ways to solve this issue, one is with python as i said before and another is the use of materialized view, that enchance the use of CLQ language but the creation of a view is a use of memory. 

The choice of how to subdivide and re-aggregate the information is subordinate to our data reading needs, that is, to the list of queries that will be executed which, therefore, must necessarily be defined before the creation of the nonrelational schema. Usually each query corresponds to a dedicated table, but in our case, since we are working with a Column-Family Database each row does not necessarily need to have values for all the columns in the table (without having to save in memory values such as **NaN** or **NULL** that would waste space) we could merge the information of several queries in the same table according to the type of subject of interest (e.g. different airports, airplanes, airlines or a couple of the prevoius subjects). This new structure allows us to eventually add new columns/information/statistics to a row/relation at a later time based on our future needs.

The new schema is now composed by the following tables:

### `Airline_Airports`

This table contains some statistics for the `1:N` relationship bewtween each `Airline` and all his `Airports`.

| CARRIER | AIRLINE | FAA | AIRPORT | NUMBER OF FLIGHTS | NUMBER OF FLIGHTS WITH RAIN | AVERAGE ARRRIVE DELAY |
| :-: | :-: | :-: | :-: | :-: | :-: | :-: |
| TEXT & PK | TEXT | TEXT & PK | TEXT | INT | INT | FLOAT | 

In [11]:
session.execute("""DROP TABLE IF EXISTS airline_airports;""") 
session.execute('''CREATE TABLE IF NOT EXISTS airline_airports (
  carrier text,
  airline text,
  faa text,
  airport text,
  n_flights int,
  n_rainy_flights int,
  avg_arr_delay float,
  PRIMARY KEY ((carrier, faa))
);
''')

### `Airline_Planes`

This table contains some statistics for the `1:N` relationship bewtween each `Airline` and all his `Planes`.

| CARRIER | AIRLINE | TAILNUM | TOTAL DISTANCE | NUMBER OF FLIGHTS | MAX NUMBER OF PASSENGERS TRANSPORTED |
| :-: | :-: | :-: | :-: | :-: | :-: | 
| TEXT & PK | TEXT | TEXT & PK | TEXT | INT | INT | 

In [12]:
session.execute("""DROP TABLE IF EXISTS airline_planes;""") 
session.execute('''CREATE TABLE IF NOT EXISTS airline_planes (
  carrier text,
  airline text,
  tailnum text,
  total_distance int,
  n_flights int,
  max_cum_passengers int,
  PRIMARY KEY ((carrier, tailnum))
);
''')

### `Arrive`

This table contains some statistics for each `Departure airport` calculated for each **month**.

| FAA | AIRPORT | MONTH | NUMBER OF FLIGHTS | NUMBER OF FLIGHTS WITH RAIN | AVERAGE ARRRIVE DELAY | BLACKLIST CARRIERS |
| :-: | :-: | :-: | :-: | :-: | :-: | :-: |
| TEXT & PK | TEXT | INT & PK | INT | INT | FLOAT | LIST OF TEXT | 

***NOTE:*** The `Blacklist carriers` column is a list of the names of all the Airlines that have a bad reputation for that destination (the reputation is based on the `average arrive delay` already calculated into the table `Airlines_Airports` so the update for future records should be calculated using that column).

In [13]:
session.execute("""DROP TABLE IF EXISTS arrive;""") 
session.execute('''CREATE TABLE IF NOT EXISTS arrive (
  faa text,
  airport text,
  month int,
  n_flights int,
  n_rainy_flights int,
  avg_delay float,
  blacklist list<text>,
  PRIMARY KEY ((faa, month))
);
''')

### `Departure`

This table contains some statistics for each `Departure airport` calculated for each **Season**.

| FAA | AIRPORT | SEASON | AVERAGE TEMPERATURE | AVERAGE PRECIPITATIONS | AVERAGE HUMIDITY | AVERAGE WIND SPEED | AVERAGE VISIBILITY | NUMBER OF CANCELED FLIGHTS | 
| :-: | :-: | :-: | :-: | :-: | :-: | :-: | :-: | :-: | 
| TEXT & PK | TEXT | TEXT & PK | FLOAT | FLOAT | FLOAT | FLOAT | FLOAT | INT |

***NOTE:*** Our database contains only flights that are departed from the three different airports of *NYC* so all the wheater misurations in the previous schema were refered to only these three airports. 

In [14]:
session.execute("""DROP TABLE IF EXISTS departure;""") 
session.execute('''CREATE TABLE IF NOT EXISTS departure (
  faa text,
  airport text,
  season text,
  avg_temp float,
  avg_precip float,
  avg_humid float,
  avg_wind_speed float,
  avg_visib float,
  n_canceled_flights float,
  PRIMARY KEY ((faa, season))
);
''')

### `Airplane`

This table contains some statistics for each `Airplane`.  

| TAILNUM | AIRPLANE INFO | YEAR | DISUSED | AIRLINE LIST | CARRIER LIST | NUMBER OF AIRLINES | TOTAL DISTANCE | AVERAGE DELAY | TIMEZONE LIST | UNIQUE DESTINATION LIST | 
| :-: | :-: | :-: | :-: | :-: | :-: | :-: | :-: | :-: | :-: | :-: | 
| TEXT & PK | DICTIONARY OF TEXT | INT | INT | LIST OF TEXT | LIST OF TEXT | INT | FLOAT | FLOAT | LIST OF TEXT | LIST OF TEXT |

***NOTE:*** Additional information of the airplane like `manifacturer`, `number of engines`, `engine type` etc... are stored together into the `airplane info` **supercolumn** which is a dictionary in which each key is the type of addiotional info and the value is the information. This solution allows us to reduce the number of columns into the table and still allows us to store differen number of information for each airplane without waste memory since each dictionary is a mutable element. This solution has the risk that it could slow down performance. However, this is not the case here since our queries never look for just one of the elements inside it, we always want all of them together. If the goal becomes to filter for individual elements within it, then it would be necessary to reassess what we want to optimize.

In [15]:
session.execute("""DROP TABLE IF EXISTS airplane_table;""") 
session.execute('''CREATE TABLE IF NOT EXISTS airplane_table (
  tailnum text,
  airplane_info map<text, text>,
  year int,
  disused int,
  airlines_list set<text>,
  carrier_list set<text>,
  n_airlines int,
  total_distance float,
  avg_delay float,
  timezone_list set<text>,
  uniq_dest_list set<text>,
  PRIMARY KEY (tailnum)
);
''')

### `Flights`

This table contains all the essentil information of a scheduled `Flight`.  

| FLIGHT ID | DEPARTURE FAA | DEPARTURE AIRPORT | ARRIVE FAA | ARRIVE AIRPORT | CARRIER | AIRLINE | TAILNUM | SCHEDULED DEPARTURE TIME | 
| :-: | :-: | :-: | :-: | :-: | :-: | :-: | :-: | :-: | 
| INT & PK | TEXT | TEXT | TEXT | TEXT | TEXT | TEXT | TEXT | TEXT | 

In [16]:
session.execute('''CREATE TABLE IF NOT EXISTS flights (
  id int,
  departure_faa text,
  departure_airport text,
  arrive_faa text,
  arrive_airport text,
  carrier text,
  airline text,
  tailnum text,
  sched_dep_time timestamp,
  PRIMARY KEY (id)
);
''')

---
###

In [8]:
# check if all the tables into the keyspace are correctly create
tables = session.execute('''DESCRIBE TABLES''')

for table in tables:
    print(table[2])

airline_airports
airline_planes
airplane_table
arrive
departure
flights


----

## <h2 style="text-align: center;">Insert of the records</h2>

----

Since we redesigned the tables of our Database by directly inserting the aggregates that interest us into the queries, we had to start from the original .csv files with the raw data and perform a long list of mergers and processing in order to obtain the records of the new tables. The Python steps performed in this step are described exhaustively in the first cell of the section delimited by the name of the corresponding new table.

For the insertion of the records we used the `session.prepared` statements which are queries that are parsed by Cassandra and then saved for later use. When the driver uses a prepared statement, it only needs to send the values of parameters to bind. This lowers network traffic and CPU utilization within Cassandra because Cassandra does not have to re-parse the query each time.

***Note:*** The placeholders for prepared statements are `?` characters instead of the usaul `%s` for non-parsed statements.

### `Airline_Airports`

In [9]:
# Merge flights data with airlines data on the 'carrier' column using a left join
temporary_df = flights.merge(airlines, on='carrier', how='left')

# Rename the 'dest' column to 'faa' in the temporary dataframe
temporary_df = temporary_df.rename(columns={'dest': 'faa'})

# Merge the updated temporary dataframe with airports data on the 'faa' column using a left join
query_df = temporary_df.merge(airports, on='faa', how='left')

# Rename the 'index' column to 'weather_index' in the weathers dataframe
weathers = weathers.rename(columns={'index': 'weather_index'})

# Merge the query_df with the weathers dataframe on the 'weather_index' column using a left join
query_final_df = query_df.merge(weathers, on='weather_index', how='left')

# Drop unnecessary columns from the final query dataframe
query_final_df = query_final_df.drop([
    'flight', 'sched_dep_time', 'sched_arr_time', 'unicode_x', 'weather_index', 
    'lon', 'alt', 'tz', 'check', 'unicode_y', 'visib', 'humid', 'pressure', 
    'wind_gust', 'wind_speed', 'air_time_min', 'lat', 'origin_y', 'temp', 
    'dewp', 'wind_dir', 'time_hour_y'], axis=1)

# Rename columns for better clarity
query_final_df = query_final_df.rename(columns={'name_x': 'airline'})
query_final_df = query_final_df.rename(columns={'name_y': 'airport'})

# Drop all the canceled flights
query_final_df = query_final_df[query_final_df['canceled'] != 1]

# Group the final query dataframe by 'airport', 'carrier', and 'airline'
grouped_df = query_final_df.groupby(['airport', 'carrier', 'airline'])

# Aggregate the grouped data to calculate:
# - Number of flights
# - Number of flights with precipitation
# - Average arrival delay
# - FAA code of the airport
airline_airports = grouped_df.agg({
    'tailnum': 'count',                                # Number of flights
    'precip': lambda x: (x > 0).sum(),                 # Number of flights with precipitation
    'arr_delay_min': 'mean',                           # Average arrival delay
    'faa': 'first'                                     # FAA code of the airport
})

# Rename the columns of the aggregated dataframe
airline_airports = airline_airports.rename(columns={
    'tailnum': 'num_flights', 
    'precip': 'num_flights_with_precip', 
    'arr_delay_min': 'avg_arrival_delay', 
    'faa': 'airport_faa'
})

# Reset the index to turn 'airport', 'carrier', and 'airline' into regular columns
airline_airports = airline_airports.reset_index()

# Sort the resulting dataframe by 'airport_faa'
airline_airports = airline_airports.sort_values(by='airport_faa')

# Output the final dataframe
airline_airports

,airport,carrier,airline,num_flights,num_flights_with_precip,avg_arrival_delay,airport_faa
2,Albuquerque International Sunport,B6,JetBlue Airways,254,15,4.38189,ABQ
195,Nantucket Mem,B6,JetBlue Airways,265,10,4.852273,ACK
1,Albany Intl,EV,ExpressJet Airlines Inc.,419,19,14.397129,ALB
298,Ted Stevens Anchorage Intl,UA,United Air Lines Inc.,8,0,-2.5,ANC
114,Hartsfield Jackson Atlanta Intl,9E,Endeavor Air Inc.,57,5,0.857143,ATL
...,...,...,...,...,...,...,...
41,Cherry Capital Airport,EV,ExpressJet Airlines Inc.,63,3,11.951613,TVC
175,Mc Ghee Tyson,9E,Endeavor Air Inc.,265,16,3.89434,TYS
176,Mc Ghee Tyson,EV,ExpressJet Airlines Inc.,314,19,41.15016,TYS
194,NW Arkansas Regional,MQ,Envoy Air,724,51,6.956215,XNA


In [19]:
def insert_airline_airports(session, df):
    query = "INSERT INTO airline_airports (carrier, airline, faa, airport, n_flights, n_rainy_flights, avg_arr_delay) VALUES (?, ?, ?, ?, ?, ?, ?)"
    prepared = session.prepare(query)
    for _, row in df.iterrows():
        # NOTE: the row-keys into the .csv files doesn't always match the column name into the tables but this is not a problem
        session.execute( prepared, (
            row['carrier'], 
            row['airline'], 
            row['airport_faa'], 
            row['airport'], 
            row['num_flights'], 
            row['num_flights_with_precip'], 
            row['avg_arrival_delay'] if not pd.isnull(row['avg_arrival_delay']) else None
            ))

insert_airline_airports(session, airline_airports)

### `Airline_Planes`

In [13]:
# Merge flights data with airlines data on the 'carrier' column using a left join
temporary_df = flights.merge(airlines, on='carrier', how='left')

# Rename the 'dest' column to 'faa' in the temporary dataframe
temporary_df = temporary_df.rename(columns={'dest': 'faa'})

# Merge the updated temporary dataframe with planes data on the 'tailnum' column using a left join
query_df2 = temporary_df.merge(planes, on='tailnum', how='left')

# Drop all the canceled flights
query_df2 = query_df2[query_df2['canceled'] != 1]

# Drop unnecessary columns from the merged dataframe
query_df2 = query_df2.drop([
    'flight', 'time_hour', 'origin', 'sched_dep_time', 'dep_delay_min', 
    'sched_arr_time', 'arr_delay_min', 'air_time_min', 'canceled', 'unicode', 
    'weather_index', 'year', 'type', 'manufacturer', 'model', 'engines', 
    'speed', 'engine', 'disused', 'model_id'], axis=1)

# Rename the 'name' column to 'airline' for clarity
query_df2 = query_df2.rename(columns={'name': 'airline'})

# Group the dataframe by 'airline' and 'tailnum'
grouped_df = query_df2.groupby(['airline', 'tailnum'])

# Aggregate the grouped data to calculate:
# - Number of flights
# - Total distance flown
# - Total number of seats
# - FAA code of the airport (first occurrence)
# - Carrier code (first occurrence)
airline_planes = grouped_df.agg({
    'date': 'count',         # Number of flights
    'distance': 'sum',       # Total distance
    'seats': 'sum',          # Total number of seats
    'faa': 'first',          # FAA code of the airport (first occurrence)
    'carrier': 'first'       # Carrier code (first occurrence)
})

# Rename the 'date' column to 'n_flights' for clarity
airline_planes = airline_planes.rename(columns={'date': 'n_flights'})

# Reset the index to turn 'airline' and 'tailnum' into regular columns
airline_planes = airline_planes.reset_index()

# Output the final dataframe
airline_planes.sort_values(by = 'distance', ascending=False)

,airline,tailnum,n_flights,distance,seats,faa,carrier
223,American Airlines Inc.,N328AA,389,929090,99195,LAX,AA
228,American Airlines Inc.,N338AA,385,923758,98175,LAX,AA
222,American Airlines Inc.,N327AA,385,910493,98175,LAX,AA
226,American Airlines Inc.,N335AA,384,907221,97920,LAX,AA
219,American Airlines Inc.,N323AA,356,842054,90780,LAX,AA
...,...,...,...,...,...,...,...
2494,SkyWest Airlines Inc.,N760SK,1,419,80,CLE,OO
2169,ExpressJet Airlines Inc.,N824AS,1,296,55,ORF,EV
2189,ExpressJet Airlines Inc.,N881AS,1,292,55,RIC,EV
2493,SkyWest Airlines Inc.,N746SK,1,229,80,IAD,OO


In [14]:
def insert_airline_planes(session, df):
    query = "INSERT INTO airline_planes (carrier, airline, tailnum, total_distance, n_flights, max_cum_passengers) VALUES (?, ?, ?, ?, ?, ?)"
    prepared = session.prepare(query)
    for _, row in df.iterrows():
        # NOTE: the row-keys into the .csv files doesn't always match the column name into the tables but this is not a problem
        session.execute( prepared, (
            row['carrier'], 
            row['airline'], 
            row['tailnum'], 
            row['distance'], 
            row['n_flights'], 
            row['seats']
            ))

insert_airline_planes(session, airline_planes)

### `Arrive`

In [11]:
# Merge flight data with airline data based on the 'carrier' column
temporary_df = flights.merge(airlines, on='carrier', how='left')

# Rename the 'dest' column to 'faa' and 'name' to 'airlines' in the temporary dataframe
temporary_df = temporary_df.rename(columns={'dest': 'faa', 'name': 'airlines'})

# Merge the temporary dataframe with airport data on the 'faa' column using a left join
query_df = temporary_df.merge(airports, on='faa', how='left')

# Rename the 'index' column to 'weather_index' in the weather dataframe
weathers = weathers.rename(columns={'index': 'weather_index'})

# Merge the query_df with weather data on the 'weather_index' column using a left join
query_final_df3 = query_df.merge(weathers, on='weather_index', how='left')

# Drop unnecessary columns from the final dataframe
query_final_df3 = query_final_df3.drop([
    'canceled', 'dst', 'tzone', 'time_hour_x', 'flight', 'sched_dep_time', 
    'sched_arr_time', 'unicode_x', 'weather_index', 'lon', 'alt', 'tz', 'check', 
    'unicode_y', 'visib', 'humid', 'pressure', 'wind_gust', 'wind_speed', 
    'air_time_min', 'lat', 'origin_y', 'temp', 'dewp', 'wind_dir', 'time_hour_y', 
    'origin_x', 'dep_delay_min'], 
    axis=1
)

# Rename the 'name' column to 'airport'
query_final_df3 = query_final_df3.rename(columns={'name': 'airport'})

# Extract the month from the 'date' column and create a new 'month' column
query_final_df3['month'] = pd.to_datetime(query_final_df3['date']).dt.month

# Group the dataframe by 'faa', 'month', and 'airport' and aggregate the data
aggregated = query_final_df3.groupby(['faa', 'month', 'airport'])
aggregated_df = aggregated.agg(
    avg_delay=('arr_delay_min', 'mean'),  # Calculate the average arrival delay
    n_flights=('tailnum', 'count'),  # Count the number of flights
    n_flights_rain=('precip', lambda x: (x > 0).sum())  # Count the number of flights with precipitation
).reset_index()

# Fill NaN values in 'avg_arrival_delay' with 0 in the airline_airports dataframe
airline_airports['avg_arrival_delay'].fillna(0, inplace=True)

#We already had the information about the average delay in the Table 'airline_airpots' so we use directly that to obtain our blacklist
#that is an information about the airlines that have a mean delay over 3 minutes
airline_airports['blacklist'] = np.where(airline_airports['avg_arrival_delay'] > 3, airline_airports['airline'], '')

# Group the airline_airports dataframe by 'airport'
black = airline_airports.groupby(['airport'])

# Aggregate the 'blacklist' column by concatenating non-empty values
black_df = black.agg({
    'blacklist': lambda x: ', '.join(filter(lambda y: y != '', x))
}).reset_index()

# Merge the aggregated_df with black_df on the 'airport' column using a left join
arrive = aggregated_df.merge(black_df, on='airport', how='left')

#We want blacklist as a list 
def string_to_list(stringa):
    return stringa.split(',')

arrive['blacklist'] = arrive['blacklist'].apply(string_to_list)

# Display the final dataframe
arrive

,faa,month,airport,avg_delay,n_flights,n_flights_rain,blacklist
0,ABQ,4,Albuquerque International Sunport,12.222222,9,0,[JetBlue Airways]
1,ABQ,5,Albuquerque International Sunport,-6.516129,31,2,[JetBlue Airways]
2,ABQ,6,Albuquerque International Sunport,2.933333,30,4,[JetBlue Airways]
3,ABQ,7,Albuquerque International Sunport,17.096774,31,3,[JetBlue Airways]
4,ABQ,8,Albuquerque International Sunport,-3.548387,31,1,[JetBlue Airways]
...,...,...,...,...,...,...,...
1107,XNA,8,NW Arkansas Regional,6.977011,89,7,"[Envoy Air, ExpressJet Airlines Inc.]"
1108,XNA,9,NW Arkansas Regional,-10.3375,83,2,"[Envoy Air, ExpressJet Airlines Inc.]"
1109,XNA,10,NW Arkansas Regional,5.032967,92,1,"[Envoy Air, ExpressJet Airlines Inc.]"
1110,XNA,11,NW Arkansas Regional,10.611111,72,6,"[Envoy Air, ExpressJet Airlines Inc.]"


In [23]:
def insert_arrive(session, df):
    query = "INSERT INTO arrive (faa, airport, month, n_flights, n_rainy_flights, avg_delay, blacklist) VALUES (?, ?, ?, ?, ?, ?, ?)"
    prepared = session.prepare(query)
    for _, row in df.iterrows():
        # NOTE: the row-keys into the .csv files doesn't always match the column name into the tables but this is not a problem
        session.execute( prepared, (
            row['faa'],
            row['airport'],
            row['month'], 
            row['n_flights'], 
            row['n_flights_rain'], 
            row['avg_delay'], 
            row['blacklist']
            ))

insert_arrive(session, arrive)

### `Departure`

In [12]:
# Merge the flights and weathers DataFrames on 'weather_index', performing a left join
temporary_df = flights.merge(weathers, on='weather_index', how='left')

# Drop unnecessary columns from the merged DataFrame
temporary_df = temporary_df.drop([
    'time_hour_y', 'check', 'unicode_y', 'pressure', 'wind_gust', 'wind_dir', 'dest',
    'sched_dep_time', 'dep_delay_min', 'sched_arr_time', 'arr_delay_min', 'weather_index',
    'time_hour_x', 'time_hour_y', 'distance', 'air_time_min', 'unicode_x', 'origin_y', 'dewp'
], axis=1)

# Convert the 'date' column to datetime format
temporary_df['date'] = pd.to_datetime(temporary_df['date'])

# Define a function to map months to seasons
def get_season(date):
    month = date.month
    if month in [12, 1, 2]:
        return 'Winter'
    elif month in [3, 4, 5]:
        return 'Spring'
    elif month in [6, 7, 8]:
        return 'Summer'
    elif month in [9, 10, 11]:
        return 'Fall'

# Apply the function to create the 'season' column
temporary_df['season'] = temporary_df['date'].apply(get_season)

# Group by 'origin_x' and 'season' and calculate the required statistics
departure = temporary_df.groupby(['origin_x', 'season']).agg(
    average_temperature = ('temp', 'mean'),         # Calculate the mean temperature
    average_precip = ('precip', 'mean'),            # Calculate the mean precipitation
    average_humidity = ('humid', 'mean'),           # Calculate the mean humidity
    average_wind_speed = ('wind_speed', 'mean'),    # Calculate the mean wind speed
    average_visibility = ('visib', 'mean'),         # Calculate the mean visibility
    n_canceled_flights = ('canceled', 'sum')        # Sum the number of canceled flights
).reset_index()

# Map airport codes to full airport names
airport_names = {
    'EWR': 'Newark Liberty International Airport',
    'LGA': 'LaGuardia Airport',
    'JFK': 'John F. Kennedy International Airport'
}
# Create a new column 'departure_airport' with the full airport names
departure['departure_airport'] = departure['origin_x'].map(airport_names)

# Print the resulting DataFrame
departure


,origin_x,season,average_temperature,average_precip,average_humidity,average_wind_speed,average_visibility,n_canceled_flights,departure_airport
0,EWR,Fall,59.008434,0.001568,58.018092,9.594728,9.600304,259,Newark Liberty International Airport
1,EWR,Spring,54.010734,0.005462,55.507164,10.773869,9.250605,746,Newark Liberty International Airport
2,EWR,Summer,78.059826,0.009188,62.035936,9.570993,9.539185,710,Newark Liberty International Airport
3,EWR,Winter,37.260333,0.004600,62.328742,10.180340,8.571112,918,Newark Liberty International Airport
4,JFK,Fall,58.806210,0.001138,59.479692,11.596398,9.562394,87,John F. Kennedy International Airport
5,JFK,Spring,51.182963,0.003202,59.492917,13.468321,9.131725,229,John F. Kennedy International Airport
6,JFK,Summer,75.779840,0.006131,67.508563,11.292707,9.420847,232,John F. Kennedy International Airport
7,JFK,Winter,37.057397,0.004049,62.063099,12.494523,8.772639,406,John F. Kennedy International Airport
8,LGA,Fall,59.336649,0.002365,54.933415,10.947700,9.567638,274,LaGuardia Airport
9,LGA,Spring,53.132122,0.004808,54.129249,12.063018,9.319460,505,LaGuardia Airport


In [25]:
def insert_departure(session, seasonal_stats):
    query = "INSERT INTO departure (faa, airport, season, avg_temp, avg_precip, avg_humid, avg_wind_speed, avg_visib, n_canceled_flights) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)"
    prepared = session.prepare(query)
    for _, row in seasonal_stats.iterrows():
        # NOTE: the row-keys into the .csv files doesn't always match the column name into the tables but this is not a problem
        session.execute( prepared, (
            row['origin_x'],
            row['departure_airport'],
            row['season'], 
            row['average_temperature'], 
            row['average_precip'], 
            row['average_humidity'], 
            row['average_wind_speed'],
            row['average_visibility'],
            row['n_canceled_flights']
            ))

insert_departure(session, departure)

### `Airplane`

In [13]:
# Merge the flights and airlines DataFrames on 'carrier', performing a left join
temporary_df = flights.merge(airlines, on='carrier', how='left')

# Rename the 'dest' column to 'faa'
temporary_df = temporary_df.rename(columns={'dest': 'faa'})

# Merge the temporary_df with the planes DataFrame on 'tailnum', performing a left join
query_df5 = temporary_df.merge(planes, on='tailnum', how='left')

# Merge the resulting DataFrame with the airports DataFrame on 'faa', performing a left join
complete_df5 = query_df5.merge(airports, on='faa', how='left')

# Drop unnecessary columns from the complete DataFrame
complete_df5 = complete_df5.drop([
    'time_hour', 'sched_dep_time', 'sched_arr_time', 'lat', 'lon', 'alt', 'tz', 'dst', 
    'canceled', 'air_time_min', 'date', 'unicode', 'model_id', 'weather_index', 'flight', 
    'origin', 'engines', 'seats', 'speed'
], axis=1)

# Rename columns for clarity
complete_df5 = complete_df5.rename(columns={'name_x': 'airlines', 'name_y': 'airport'})

# Group by 'tailnum' and calculate the required statistics
airplane_table = complete_df5.groupby('tailnum').agg(
    manufacturer=('manufacturer', 'first'),                 # Get the first value of 'manufacturer'
    model=('model', 'first'),                               # Get the first value of 'model'
    type=('type', 'first'),                                 # Get the first value of 'type'
    year=('year', 'first'),                                 # Get the first value of 'year'
    disused=('disused', 'first'),                           # Get the first value of 'disused'
    airline_list=('airlines', lambda x: list(x.unique())),  # Create a list of unique airlines
    carrier_list=('carrier', lambda x: list(x.unique())),   # Create a list of unique carriers
    n_airlines=('airlines', 'nunique'),                     # Count the number of unique airlines
    n_flights=('tailnum', 'count'),                         # Count the number of flights
    n_timezone_list=('tzone', lambda x: list(x.unique())),  # Create a list of unique timezones
    uniq_dest_list=('faa', lambda x: list(x.unique())),     # Create a list of unique destinations
    total_distance=('distance', 'sum'),                     # Sum the total distance
    avg_delay=('arr_delay_min', 'mean')                     # Calculate the average arrival delay
).reset_index()

# Print the resulting DataFrame
airplane_table


,tailnum,manufacturer,model,type,year,disused,airline_list,carrier_list,n_airlines,n_flights,n_timezone_list,uniq_dest_list,total_distance,avg_delay
0,D942DN,None,None,None,<NA>,1,[Delta Air Lines Inc.],[DL],1,4,[America/New_York],"[ATL, MCO]",3418,31.5
1,N0EGMQ,None,None,None,<NA>,1,[Envoy Air],[MQ],1,371,"[America/New_York, America/Chicago]","[CLT, ATL, DTW, MSP, BNA, ORD, CMH, CLE, TPA, ...",250866,9.982955
2,N10156,EMBRAER,EMB-145XR,Fixed wing multi engine,2004,0,[ExpressJet Airlines Inc.],[EV],1,153,"[America/New_York, America/Chicago]","[PIT, CHS, MSP, CMH, MCI, PWM, DTW, CVG, MHT, ...",115966,12.717241
3,N102UW,AIRBUS INDUSTRIE,A320-214,Fixed wing multi engine,1998,0,[US Airways Inc.],[US],1,48,[America/New_York],[CLT],25722,2.9375
4,N103US,AIRBUS INDUSTRIE,A320-214,Fixed wing multi engine,1999,0,[US Airways Inc.],[US],1,46,[America/New_York],[CLT],24619,-6.934783
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4038,N997DL,MCDONNELL DOUGLAS AIRCRAFT CO,MD-88,Fixed wing multi engine,1992,0,[Delta Air Lines Inc.],[DL],1,63,"[America/New_York, America/Chicago]","[MIA, PBI, TPA, RSW, MCO, DTW, ATL, BOS, FLL, ...",54669,4.903226
4039,N998AT,BOEING,717-200,Fixed wing multi engine,2002,0,[AirTran Airways Corporation],[FL],1,26,[America/New_York],"[CAK, ATL]",15432,29.96
4040,N998DL,MCDONNELL DOUGLAS CORPORATION,MD-88,Fixed wing multi engine,1992,0,[Delta Air Lines Inc.],[DL],1,77,"[America/New_York, America/Chicago]","[DTW, MCO, ATL, MSP, BOS, FLL, MIA, PBI, TPA, ...",66052,16.394737
4041,N999DN,MCDONNELL DOUGLAS CORPORATION,MD-88,Fixed wing multi engine,1992,0,[Delta Air Lines Inc.],[DL],1,61,"[America/New_York, America/Chicago]","[PBI, MCO, FLL, ATL, BOS, MIA, MSP, DTW, TPA, ...",54623,14.311475


In [27]:
def insert_airplane(session, df):
    query = "INSERT INTO airplane_table (tailnum, airplane_info,year, disused, airlines_list,carrier_list,n_airlines, total_distance, avg_delay,  timezone_list, uniq_dest_list) VALUES (?, ?,?, ?,?, ?, ?, ?, ?,?, ?)"
    prepared = session.prepare(query)
    for _, row in df.iterrows():
        # NOTE: the row-keys into the .csv files doesn't always match the column name into the tables but this is not a problem
        session.execute( prepared, (
            row['tailnum'],
            {'manufacturer': row['manufacturer'], 'model': row['model'], 'type': row['type']},
            row['year'] if not pd.isnull(row['year']) else None,
            row['disused'],
            row['airline_list'], 
            row['carrier_list'], 
            row['n_airlines'], 
            row['total_distance'],
            row['avg_delay']if not pd.isnull(row['avg_delay']) else None, 
            row['n_timezone_list'],
            row['uniq_dest_list']
            ))

insert_airplane(session, airplane_table)

### `Flights`

In [14]:
# Assign the flights DataFrame to a temporary DataFrame
temporary_df = flights

# Rename the 'dest' column to 'faa' for merging purposes
temporary_df = temporary_df.rename(columns={'dest': 'faa'})

# Merge the temporary DataFrame with the airports DataFrame on 'faa', performing a left join
query_df6 = temporary_df.merge(airports, on='faa', how='left')

# Define a dictionary to map airport codes to full airport names
airport_names = {
    'EWR': 'Newark Liberty International Airport',
    'LGA': 'LaGuardia Airport',
    'JFK': 'John F. Kennedy International Airport'
}

# Create a new column 'departure_airport' by mapping the 'origin' column to the full airport names
query_df6['departure_airport'] = query_df6['origin'].map(airport_names)

# Drop unnecessary columns from the DataFrame
query_df6 = query_df6.drop([
    'lat', 'alt', 'tz', 'dst', 'tzone', 'unicode', 'weather_index', 'lon', 
    'air_time_min', 'dep_delay_min', 'sched_arr_time', 'arr_delay_min', 
    'distance', 'canceled'
], axis=1)

# Rename the 'name' column to 'arrive_airport' for clarity
query_df6 = query_df6.rename(columns={'name': 'arrive_airport'})

# Merge the resulting DataFrame with the airlines DataFrame on 'carrier', performing a left join
new_flights_table = query_df6.merge(airlines, on='carrier', how='left')

# Rename columns for clarity
new_flights_table = new_flights_table.rename(columns={
    'name': 'airlines', 
    'origin': 'departure_faa', 
    'faa': 'arrive_faa'
})

# Convert the 'sched_dep_time' column to string format
new_flights_table['sched_dep_time'] = new_flights_table['sched_dep_time'].astype(str)

# Convert the 'sched_dep_time' column to datetime64 format
new_flights_table['sched_dep_time'] = pd.to_datetime(new_flights_table['sched_dep_time'])

# Print the resulting DataFrame
new_flights_table


,flight,carrier,tailnum,time_hour,departure_faa,arrive_faa,sched_dep_time,date,arrive_airport,departure_airport,airlines
0,1,UA,N14228,2013-01-01 05:00:00,EWR,IAH,2013-01-01 05:15:00,2013-01-01,George Bush Intercontinental,Newark Liberty International Airport,United Air Lines Inc.
1,2,UA,N24211,2013-01-01 05:00:00,LGA,IAH,2013-01-01 05:29:00,2013-01-01,George Bush Intercontinental,LaGuardia Airport,United Air Lines Inc.
2,3,AA,N619AA,2013-01-01 05:00:00,JFK,MIA,2013-01-01 05:40:00,2013-01-01,Miami Intl,John F. Kennedy International Airport,American Airlines Inc.
3,4,B6,N804JB,2013-01-01 05:00:00,JFK,BQN,2013-01-01 05:45:00,2013-01-01,Raphael Hernàndez International Airport,John F. Kennedy International Airport,JetBlue Airways
4,5,DL,N668DN,2013-01-01 06:00:00,LGA,ATL,2013-01-01 06:00:00,2013-01-01,Hartsfield Jackson Atlanta Intl,LaGuardia Airport,Delta Air Lines Inc.
...,...,...,...,...,...,...,...,...,...,...,...
334259,334260,B6,N516JB,2013-09-30 23:00:00,JFK,PSE,2013-09-30 23:59:00,2013-09-30,Mercedita International Airport,John F. Kennedy International Airport,JetBlue Airways
334260,334261,EV,N740EV,2013-09-30 18:00:00,LGA,BNA,2013-09-30 18:42:00,2013-09-30,Nashville Intl,LaGuardia Airport,ExpressJet Airlines Inc.
334261,334262,MQ,N535MQ,2013-09-30 12:00:00,LGA,BNA,2013-09-30 12:10:00,2013-09-30,Nashville Intl,LaGuardia Airport,Envoy Air
334262,334263,MQ,N511MQ,2013-09-30 11:00:00,LGA,CLE,2013-09-30 11:59:00,2013-09-30,Cleveland Hopkins Intl,LaGuardia Airport,Envoy Air


In [82]:
def insert_flights(session, df):
    query = "INSERT INTO flights (id, departure_faa, departure_airport, arrive_faa, arrive_airport, carrier, airline, tailnum, sched_dep_time) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)"
    prepared = session.prepare(query)
    for _, row in df.iterrows():
        # NOTE: the row-keys into the .csv files doesn't always match the column name into the tables but this is not a problem
        session.execute( prepared, (
            row['flight'],
            row['departure_faa'],
            row['departure_airport'],
            row['arrive_faa'], 
            row['arrive_airport'], 
            row['carrier'], 
            row['airlines'], 
            row['tailnum'],
            row['sched_dep_time']
            ))

insert_flights(session, new_flights_table)

----

## <h2 style="text-align: center;">Queries execution</h2>

----

#### Q1

*The most common route (faa + name) for each airline (name + carrier) considering only the departed and not cancelled flights + the number of flights to that destination from that airline.*

Here we can see the solution with a materialized view for extracting a maximum. We chose to use it here because even though our other queries didn't require the use of this materialized view, we thought that it could be helpful in a real environment that uses this type of database. 


***NOTE:*** Here we can also notice a limitation of materialized views. Unfortunately, due to the fact that they are still being implemented in Cassandra, they can sometimes be inconsistent. In fact, executing this query multiple times may result in the first element not being displayed. The result remains the same, but the first element is omitted. There is a list of known bugs available at the following link: 

https://issues.apache.org/jira/projects/CASSANDRA/issues/CASSANDRA-15071?filter=allopenissues



In [18]:
# Execute a query to get distinct carriers and their FAA codes from the airline_airports table
carriers = session.execute('''SELECT DISTINCT carrier, faa FROM airline_airports;''')

session.execute('''DROP MATERIALIZED VIEW IF EXISTS airline_airports_by_flights ''')
session.execute('''CREATE MATERIALIZED VIEW IF NOT EXISTS airline_airports_by_flights AS
  SELECT carrier, airline, faa, airport, n_flights
  FROM airline_airports
  WHERE carrier IS NOT NULL AND n_flights IS NOT NULL AND faa IS NOT NULL
  PRIMARY KEY ((carrier), n_flights, faa)
  WITH CLUSTERING ORDER BY (n_flights DESC, faa ASC);
''')

# Prepare the sub-query to get the maximum number of flights for each carrier
sub_query = session.prepare('''SELECT carrier, airline, faa, airport, n_flights
                                      FROM airline_airports_by_flights
                                      WHERE carrier = ?
                                      LIMIT 1;''')

# Initialize a dictionary to store the results
max_per_carrier = dict()

# Extract the distinct carriers and execute the sub-query for each carrier
for carrier in set(row.carrier for row in carriers):
    # Execute the prepared sub-query for the current carrier
    result = session.execute(sub_query, [carrier]).one()
    # If a result is found, store it in the dictionary
    if result:
        max_per_carrier[carrier] = {
            'Airline': result.airline, 
            'faa': result.faa, 
            'Airport': result.airport, 
            'Number of flights': result.n_flights
        }

# Convert the dictionary with results to a pandas DataFrame
ResultQ1 = pd.DataFrame(max_per_carrier)

# Set the index name to 'Carrier'
ResultQ1.index.name = 'Carrier'

# Transpose the DataFrame so that carriers are the columns
ResultQ1 = ResultQ1.transpose()

# Output the final DataFrame
ResultQ1.sort_values(by = 'Number of flights', ascending = False)


Carrier,Airline,faa,Airport,Number of flights
DL,Delta Air Lines Inc.,ATL,Hartsfield Jackson Atlanta Intl,10479
US,US Airways Inc.,CLT,Charlotte Douglas Intl,8508
AA,American Airlines Inc.,DFW,Dallas Fort Worth Intl,7019
UA,United Air Lines Inc.,IAH,George Bush Intercontinental,6831
MQ,Envoy Air,RDU,Raleigh Durham Intl,4554
WN,Southwest Airlines Co.,MDW,Chicago Midway Intl,4044
EV,ExpressJet Airlines Inc.,IAD,Washington Dulles Intl,3829
B6,JetBlue Airways,SJU,Luis Mugnoz Marin International Airport,2718
VX,Virgin America,LAX,Los Angeles Intl,2561
FL,AirTran Airways Corporation,ATL,Hartsfield Jackson Atlanta Intl,2289


#### Q2:

*For each airline company (carrier + name) return the Airplane (tail_num) that has the highest cumulative flight distance with that company + return the number of non canceled flights ad the maximun number of passengers that the airplane can have have trasported.*

Here instead there is the solution with Python for the problem of extract the maximum.

In [20]:
# Execute a query to get distinct carriers and their FAA codes from the airline_airports table
carriers = session.execute('''SELECT DISTINCT carrier, faa FROM airline_airports;''')

# Prepare the sub-query to get the maximum total distance for each carrier
max_distance_query = session.prepare('''SELECT carrier, MAX(total_distance) as max_total_distance 
                                        FROM airline_planes 
                                        WHERE carrier = ? 
                                        ALLOW FILTERING;''')
# Initialize a dictionary to store the max total distance for each carrier
max_distance_per_carrier = dict()

# Execute the sub-query for each carrier
for carrier in set(row.carrier for row in carriers):
    result = session.execute(max_distance_query, [carrier]).one()
    if result:
        max_distance_per_carrier[carrier] = result.max_total_distance

# Initialize a dictionary to store the results
max_per_carrier = dict()

# Prepare the query to get the details of the plane with the max total distance
details_query = session.prepare('''SELECT carrier, airline, tailnum, total_distance, n_flights, max_cum_passengers
                                   FROM airline_planes 
                                   WHERE carrier = ? AND total_distance = ? 
                                   ALLOW FILTERING;''')

# Execute the query for each carrier using the max total distance obtained in the previous step
for carrier, max_distance in max_distance_per_carrier.items():
    result = session.execute(details_query, [carrier, max_distance]).one()
    if result:
        max_per_carrier[carrier] = {
            'Airline': result.airline,
            'Plane tailnum': result.tailnum,
            'Total distance': result.total_distance,
            'Number of Flights': result.n_flights,
            'Max number of passengers transported': result.max_cum_passengers
        }

# Convert the dictionary with results to a pandas DataFrame
ResultQ4 = pd.DataFrame(max_per_carrier)

# Set the index name to 'Carrier'
ResultQ4.index.name = 'Carrier'

# Transpose the DataFrame so that carriers are the columns
ResultQ4 = ResultQ4.transpose()

# Output the final DataFrame
ResultQ4.sort_values(by = 'Total distance', ascending = False)


Carrier,Airline,Plane tailnum,Total distance,Number of Flights,Max number of passengers transported
AA,American Airlines Inc.,N328AA,929090,389,99195
DL,Delta Air Lines Inc.,N718TW,772906,327,58206
UA,United Air Lines Inc.,N502UA,726276,286,50908
B6,JetBlue Airways,N789JB,527031,331,66200
VX,Virgin America,N638VA,389991,156,28392
MQ,Envoy Air,N725MQ,306006,546,0
HA,Hawaiian Airlines Inc.,N380HA,199320,40,15080
9E,Endeavor Air Inc.,N922XJ,188277,309,29355
EV,ExpressJet Airlines Inc.,N15980,153461,305,16775
US,US Airways Inc.,N676AW,124998,78,15600


#### Q3

*For each destination airport in FLIGHTS return the month with the highest number of flights and it's flight count*

***NOTE:*** No duplicates were allowed thanks to the aggregation so the results is slightly different from the one in SQL

In [24]:
# Extraction of the distinct couple of airports and months
distinct_airports = session.execute('''SELECT DISTINCT faa, month FROM arrive;''')

# Prepare the query to obtain the maximum number of flights per carrier for each airport
sub_query = session.prepare('''SELECT faa, airport, month, MAX(n_flights) as max_flights FROM arrive WHERE faa = ? ALLOW FILTERING;''')

# Initialize a dictionary to store the results
max_per_airport = dict()

# Extract the distinct FAA codes and execute the sub-query for each airport
for airport in set(row.faa for row in distinct_airports):

    result = session.execute(sub_query, [airport]).one()
    # If there is a result, add it to the dictionary
    if result:
        max_per_airport[airport] = {'airport info': result.airport, 'month': result.month, 'max_flights': result.max_flights}

# Convert the dictionary to a pandas DataFrame
ResultQ3 = pd.DataFrame(max_per_airport)
ResultQ3.index.name = 'faa'

# Transpose the DataFrame to have FAA codes as rows
ResultQ3 = ResultQ3.transpose()

# Sort the DataFrame by 'max_flights' in descending order
ResultQ3.sort_values(by='max_flights', ascending=False)

faa,airport info,month,max_flights
ORD,Chicago Ohare Intl,10,1594
ATL,Hartsfield Jackson Atlanta Intl,7,1511
LAX,Los Angeles Intl,6,1503
BOS,General Edward Lawrence Logan Intl,5,1353
CLT,Charlotte Douglas Intl,3,1293
...,...,...,...
HDN,Yampa Valley,12,5
MTJ,Montrose Regional Airport,3,5
PSP,Palm Springs Intl,1,5
ANC,Ted Stevens Anchorage Intl,7,4


#### Q4

*Return the average weather condition (temp, humid, precipitation, wind_speed, visibility) and the number of canceled flights for each departure airport grouped by season and the number of canceled flights.*

This query requested very specific information and required aggregation in a particular way. To avoid impacting other tables and optimize the retrieval of this result, we decided to create a separate table dedicated solely to this query. This approach ensures that the performance of other tables and queries is not affected, and it streamlines the process of obtaining the desired result efficiently. Also because it can be interesting to know this type of statistics to have an idea of the weather condition of a place

In [25]:
ResultQ4 = session.execute('''select * from departure''' )

# Print and sort by 'season' the result of query 4
ResultQ4 = pd.DataFrame(ResultQ4)
ResultQ4.sort_values(by = 'season')

,faa,season,airport,avg_humid,avg_precip,avg_temp,avg_visib,avg_wind_speed,n_canceled_flights
2,LGA,Fall,LaGuardia Airport,54.933414,0.002365,59.336651,9.567637,10.947700,274.0
4,JFK,Fall,John F. Kennedy International Airport,59.479694,0.001138,58.806210,9.562394,11.596398,87.0
11,EWR,Fall,Newark Liberty International Airport,58.018093,0.001568,59.008434,9.600304,9.594728,259.0
0,EWR,Spring,Newark Liberty International Airport,55.507164,0.005462,54.010735,9.250605,10.773869,746.0
5,JFK,Spring,John F. Kennedy International Airport,59.492916,0.003202,51.182964,9.131725,13.468321,229.0
8,LGA,Spring,LaGuardia Airport,54.129250,0.004808,53.132122,9.319460,12.063018,505.0
1,EWR,Summer,Newark Liberty International Airport,62.035934,0.009188,78.059830,9.539186,9.570992,710.0
6,JFK,Summer,John F. Kennedy International Airport,67.508560,0.006131,75.779839,9.420847,11.292707,232.0
7,LGA,Summer,LaGuardia Airport,57.865040,0.006232,77.778008,9.622469,10.064643,765.0
3,EWR,Winter,Newark Liberty International Airport,62.328743,0.004600,37.260334,8.571112,10.180340,918.0


#### Q5

*Return the Airplanes with (tailnum + manufacturer, model, type) of all the airplanes that have flight with more that one airline company, the number of company which that plane has flight and their name.*

Here there are some statistics on some specific planes 

In [47]:
ResultQ5 = session.execute('''SELECT tailnum, airplane_info, airlines_list, carrier_list, n_airlines FROM airplane_table WHERE n_airlines > 1 ALLOW FILTERING;''')

# Initialize an empty list to store the processed rows
data_q5 = []

# Process each row in the query result
for row in ResultQ5:
    # Convert the OrderedMapSerializedKey to a dictionary
    airplane_info_dict = dict(row.airplane_info)
    
    # Create a dictionary for the DataFrame row
    row_data = {
        'tailnum': row.tailnum,
        'manufacturer': airplane_info_dict.get('manufacturer'),
        'model': airplane_info_dict.get('model'),
        'type': airplane_info_dict.get('type'),
        'airlines_list': list(row.airlines_list),
        'carrier_list': list(row.carrier_list),
        'n_airlines': row.n_airlines
    }
    
    # Append the dictionary to the data list
    data_q5.append(row_data)

# Create the DataFrame from the list of dictionaries
pd.DataFrame(data_q5)


,tailnum,manufacturer,model,type,airlines_list,carrier_list,n_airlines
0,N994AT,BOEING,717-200,Fixed wing multi engine,"[AirTran Airways Corporation, Delta Air Lines ...","[DL, FL]",2
1,N197PQ,BOMBARDIER INC,CL-600-2D24,Fixed wing multi engine,"[Endeavor Air Inc., ExpressJet Airlines Inc.]","[9E, EV]",2
2,N232PQ,BOMBARDIER INC,CL-600-2D24,Fixed wing multi engine,"[Endeavor Air Inc., ExpressJet Airlines Inc.]","[9E, EV]",2
3,N978AT,BOEING,717-200,Fixed wing multi engine,"[AirTran Airways Corporation, Delta Air Lines ...","[DL, FL]",2
4,N933AT,BOEING,717-200,Fixed wing multi engine,"[AirTran Airways Corporation, Delta Air Lines ...","[DL, FL]",2
5,N981AT,BOEING,717-200,Fixed wing multi engine,"[AirTran Airways Corporation, Delta Air Lines ...","[DL, FL]",2
6,N181PQ,BOMBARDIER INC,CL-600-2D24,Fixed wing multi engine,"[Endeavor Air Inc., ExpressJet Airlines Inc.]","[9E, EV]",2
7,N990AT,BOEING,717-200,Fixed wing multi engine,"[AirTran Airways Corporation, Delta Air Lines ...","[DL, FL]",2
8,N979AT,BOEING,717-200,Fixed wing multi engine,"[AirTran Airways Corporation, Delta Air Lines ...","[DL, FL]",2
9,N200PQ,BOMBARDIER INC,CL-600-2D24,Fixed wing multi engine,"[Endeavor Air Inc., ExpressJet Airlines Inc.]","[9E, EV]",2


#### Q6

*Return the 25 most old airplanes still in service (disused = 0) with some statistics like:*
- *the year of construction* 
   -  *exlude all the planes with NULL in the year field*
- *total number of flights* 
- *total distance* 
- *average arrive delay*
- *number of different airports in which that airplane has landed*
- *number of different time zone in which that airplane has landed*

***NOTE:*** Given that the years column is not a **primary key** and cannot be used as a **clustering column** (otherwise it would group together all the planes with the same year) we must sort using pandas and consequently also use python for the limit statement.

In [48]:
ResultQ6 = session.execute('''SELECT tailnum, airplane_info, year, total_distance, avg_delay, timezone_list, uniq_dest_list FROM airplane_table WHERE disused = 0 ALLOW FILTERING;''')

# Initialize an empty list to store the processed rows
data_q6 = []

# Process each row in the query result
for row in ResultQ6:
    # Convert the OrderedMapSerializedKey to a dictionary
    airplane_info_dict = dict(row.airplane_info)
    
    # Create a dictionary for the DataFrame row
    row_data = {
        'tailnum': row.tailnum,
        'manufacturer': airplane_info_dict.get('manufacturer'),
        'model': airplane_info_dict.get('model'),
        'type': airplane_info_dict.get('type'),
        'year': row.year,
        'total_distance': row.total_distance,
        'avg_delay': row.avg_delay,
        'timezone_list': list(row.timezone_list),
        'uniq_dest_list': list(row.uniq_dest_list)
    }
    
    # Append the dictionary to the data list
    data_q6.append(row_data)

# Create the DataFrame from the list of dictionaries
# Sort the DataFrame by 'year' and display the first 25 rows
pd.DataFrame(data_q6).sort_values(by='year').head(25)

,tailnum,manufacturer,model,type,year,total_distance,avg_delay,timezone_list,uniq_dest_list
1328,N381AA,DOUGLAS,DC-7BF,Fixed wing multi engine,1956.0,44776.0,3.500000,"[America/Los_Angeles, America/New_York]","[MIA, SAN, SFO]"
182,N567AA,DEHAVILLAND,OTTER DHC-3,Fixed wing single engine,1959.0,58817.0,-4.295082,[America/Chicago],"[DFW, ORD, STL]"
577,N201AA,CESSNA,150,Fixed wing single engine,1959.0,54638.0,14.117647,[America/Chicago],"[DFW, ORD, STL]"
1554,N575AA,CESSNA,210-5(205),Fixed wing single engine,1963.0,44488.0,-1.921053,[America/Chicago],"[DFW, ORD, STL]"
2210,N378AA,CESSNA,172E,Fixed wing single engine,1963.0,23139.0,12.000000,"[America/Los_Angeles, America/New_York]","[MIA, SFO]"
3173,N14629,BOEING,737-524,Fixed wing multi engine,1965.0,5664.0,-16.250000,[America/Chicago],[IAH]
764,N615AA,BEECH,65-A90,Fixed wing multi engine,1967.0,29173.0,4.045455,"[America/Denver, America/Los_Angeles, America/...","[EGE, LAS, MCO, MIA, SJU]"
3286,N425AA,PIPER,PA-28-180,Fixed wing single engine,1968.0,41964.0,-1.658537,[America/Chicago],"[DFW, ORD, STL]"
953,N383AA,BEECH,E-90,Fixed wing multi engine,1972.0,48043.0,5.666667,"[America/Los_Angeles, America/New_York]","[MIA, SAN, SFO]"
2680,N364AA,CESSNA,310Q,Fixed wing multi engine,1973.0,48847.0,-2.227273,"[America/Los_Angeles, America/New_York]","[MIA, SAN, SFO]"


#### Q7

*Return for each destination airport the average arrive delay in that airport and the airlines companies that have an average arrive delay bigger than 3 in that airport.*

***NOTE:*** The Python algorithm to calculate the mean cannot handle NaN values, so we replace all NaN values with 0 before calculating the average delay instead of dropping that specific flight. As a result, the average delay is slightly lower compared to the result obtained in SQL.

In [21]:
# Extraction of the distinct couple of airports and months
distinct_airports = session.execute('''SELECT DISTINCT faa, month FROM arrive;''')

# Prepare the query to obtain the average delay and blacklist status for each airport
sub_query = session.prepare('''SELECT faa, airport, avg(avg_delay) as avg_delay, blacklist FROM arrive WHERE faa = ? ALLOW FILTERING''')

# Initialize a dictionary to store the results
avg_delay_airports = dict()

# This loop iterates over each distinct FAA code obtained from the first query.
for airport in set(row.faa for row in distinct_airports):

    result = session.execute(sub_query, [airport]).one()
    # If there is a result, add it to the dictionary
    if result:
        avg_delay_airports[airport] = {'airport': result.airport, 'avg_delay': result.avg_delay, 'blacklist': result.blacklist}

# Convert the dictionary to a pandas DataFrame
ResultQ7 = pd.DataFrame(avg_delay_airports)
ResultQ7.index.name = 'FAA'

# Transpose the DataFrame to have FAA codes as rows
ResultQ7 = ResultQ7.transpose()

# Sort the DataFrame by 'airport'
ResultQ7.sort_values(by='airport')

FAA,airport,avg_delay,blacklist
CAK,Akron Canton Regional Airport,18.955748,[AirTran Airways Corporation]
ALB,Albany Intl,10.326901,[ExpressJet Airlines Inc.]
ABQ,Albuquerque International Sunport,4.933413,[JetBlue Airways]
AVL,Asheville Regional Airport,7.3075,[ExpressJet Airlines Inc.]
AUS,Austin Bergstrom Intl,6.240582,"[American Airlines Inc., United Air Lines Inc..."
...,...,...,...
OKC,Will Rogers World,31.406246,[ExpressJet Airlines Inc.]
HOU,William P Hobby,7.349119,"[JetBlue Airways, Southwest Airlines Co.]"
ILM,Wilmington Intl,3.891971,[ExpressJet Airlines Inc.]
HDN,Yampa Valley,4.533333,[]


#### Q8

*Return for all the flights (origin) that goes to a certan destination in a certan range of time and the airline that offer that flight with the date and the hour-minute of the flight.*

We think that this is the most utilized query in a database about flights

In [50]:
# Execute the query to select flights arriving at LAX within a specific date range
ResultQ8 = session.execute('''SELECT departure_faa AS Origin, airline, carrier, sched_dep_time FROM flights WHERE sched_dep_time >= '2013-06-01 00:00:00' AND sched_dep_time <= '2013-06-07 23:59:59' AND arrive_faa = 'LAX' ALLOW FILTERING;''')

# Convert the query result to a pandas DataFrame
ResultQ8 = pd.DataFrame(ResultQ8)

# Sort the DataFrame by 'sched_dep_time'
ResultQ8 = ResultQ8.sort_values(by='sched_dep_time')

# Display the sorted DataFrame
ResultQ8

,origin,airline,carrier,sched_dep_time
42,JFK,United Air Lines Inc.,UA,2013-06-01 06:00:00
149,EWR,United Air Lines Inc.,UA,2013-06-01 06:00:00
109,JFK,JetBlue Airways,B6,2013-06-01 07:00:00
159,JFK,Virgin America,VX,2013-06-01 07:15:00
15,JFK,Delta Air Lines Inc.,DL,2013-06-01 07:15:00
...,...,...,...,...
225,JFK,United Air Lines Inc.,UA,2013-06-07 20:25:00
256,JFK,JetBlue Airways,B6,2013-06-07 20:30:00
144,EWR,United Air Lines Inc.,UA,2013-06-07 20:55:00
254,JFK,Delta Air Lines Inc.,DL,2013-06-07 21:00:00


#### Q9

*Return the for each destination aiport and airline the count of flights affected by rain during the departure.*

For this query we have slightly difference with the output obtained with the same query in SQL. The difference is due to the fact that in this table (`Airline_airports`) contain the statistics for all the (airline, airports) pair without considering the canceled flights (which were also considered into the original Q9). 

In [51]:
ResultQ9 = session.execute('''SELECT carrier, airline, faa, airport, n_rainy_flights FROM airline_airports WHERE n_rainy_flights > 0  ALLOW FILTERING;''')

# Convert the results into a pandas dataframe and sort them by number of rainy flights
ResultQ9 = pd.DataFrame(ResultQ9)
ResultQ9.sort_values(by = 'n_rainy_flights', ascending = False)

,carrier,airline,faa,airport,n_rainy_flights
56,DL,Delta Air Lines Inc.,ATL,Hartsfield Jackson Atlanta Intl,703
33,US,US Airways Inc.,CLT,Charlotte Douglas Intl,555
22,AA,American Airlines Inc.,DFW,Dallas Fort Worth Intl,475
85,UA,United Air Lines Inc.,SFO,San Francisco Intl,467
53,UA,United Air Lines Inc.,IAH,George Bush Intercontinental,464
...,...,...,...,...,...
206,UA,United Air Lines Inc.,MTJ,Montrose Regional Airport,1
219,9E,Endeavor Air Inc.,AVL,Asheville Regional Airport,1
140,DL,Delta Air Lines Inc.,STT,Cyril E. King Airport,1
92,DL,Delta Air Lines Inc.,EYW,Key West Intl,1


#### Q10

*Returns all the destination airports that have a difference in the number of departures between the first half and the second half of each year.* 

***NOTE:***  the database contains only records for the year 2013 so that column of the query in this case is useless, but is possible to add it and let this table able to do for all the possible years.

In [52]:
# Execute a query to get distinct FAA codes and months from the arrive table
distinct_airports = session.execute('''SELECT DISTINCT faa, month FROM arrive;''')

# Prepare the first half query to get the cumulative number of flights for each airport in the first half of the year (January to June)
first_half_query = session.prepare('''SELECT faa, airport, SUM(n_flights) AS cum_flights 
                                      FROM arrive 
                                      WHERE faa = ? AND month < 7 
                                      ALLOW FILTERING''')

# Prepare the second half query to get the cumulative number of flights for each airport in the second half of the year (July to December)
second_half_query = session.prepare('''SELECT faa, airport, SUM(n_flights) AS cum_flights 
                                       FROM arrive 
                                       WHERE faa = ? AND month > 6 
                                       ALLOW FILTERING''')

# Initialize a dictionary to store the results
delta_airports = dict()

# Extract the distinct FAA codes and execute the sub-queries for each airport
for airport in set(row.faa for row in distinct_airports):
    # Execute the first half query for the current airport
    first = session.execute(first_half_query, [airport]).one()
    # Execute the second half query for the current airport
    second = session.execute(second_half_query, [airport]).one()
    # If results are found, store the data in the dictionary
    if first and second:
        delta_airports[airport] = {
            'Airport info': first.airport, 
            'First_part flights': first.cum_flights, 
            'Second_part flights': second.cum_flights, 
            'Delta': (second.cum_flights - first.cum_flights)
        }

# Convert the dictionary with results to a pandas DataFrame
ResultQ10 = pd.DataFrame(delta_airports)

# Set the index name to 'Carrier'
ResultQ10.index.name = 'Carrier'

# Transpose the DataFrame so that FAA codes are the columns
ResultQ10 = ResultQ10.transpose()

# Return only the rows that have a higher number of flights into the firts part of the year
ResultQ10 = ResultQ10[ ResultQ10['Delta'] < 0]

# Output the final DataFrame
ResultQ10.sort_values(by = 'Delta')

Carrier,Airport info,First_part flights,Second_part flights,Delta
FLL,Fort Lauderdale Hollywood Intl,6391,5645,-746
RSW,Southwest Florida Intl,2066,1461,-605
DCA,Ronald Reagan Washington Natl,4940,4477,-463
PBI,Palm Beach Intl,3460,3081,-379
BWI,Baltimore Washington Intl,1058,686,-372
PWM,Portland Intl Jetport,1361,991,-370
DFW,Dallas Fort Worth Intl,4489,4182,-307
RDU,Raleigh Durham Intl,4183,3936,-247
CLE,Cleveland Hopkins Intl,2382,2140,-242
BTV,Burlington Intl,1412,1177,-235


----

## <h2 style="text-align: center;">Insert new data + Updating Tables</h2>

----

Given that we have separate tables, it's important that when we insert new records, the aggregates of the various features related to this update are also updated coherently to ensure that the database remains consistent despite its denormalized structure. To achieve this, it's possible to automatize the insertion of new data and automatize the update of each aggregate that is influenced by it.

Here we have created an example:

We have created an interface to update the `Flights` table and all the `number_of_flights` columns of the various aggregates. This process could be replicated for every feature with the creation of appropriate widgets that handle the insertion.

***NOTE:*** When we perform an update, we are actually overwriting the value of that specific row. This is because only columns of type `counter` can be updated, and if one variable is of type counter, all of them will be.

In [23]:
# Retrive distinct airport codes 
distinct_airports = session.execute('''SELECT DISTINCT faa, month FROM arrive;''')

# Convert the result to a list of distinct airports
distinct_airports = [row.faa for row in distinct_airports]

# Retrive distinct carrier codes 
distinct_carriers = session.execute('''SELECT DISTINCT carrier, faa FROM airline_airports;''')

# Convert the result to a list of distinct carriers
distinct_carriers = [row.carrier for row in distinct_carriers]

# Retrive distinct tail numbers 
distinct_tailnums = session.execute('''SELECT DISTINCT tailnum FROM airplane_table;''')

# Convert the result to a list of distinct tail numbers
distinct_tailnums = [row.tailnum for row in distinct_tailnums]

# Function to generate a unique ID
def generate_unique_id():
    existing_ids = session.execute('''SELECT DISTINCT id FROM flights;''')
    existing_ids = [row.id for row in existing_ids]
    # Generate a new ID by incrementing the maximum existing ID
    new_id = max(existing_ids) + 1  
    return new_id

# Callback function to handle the submission of new data
def on_submit_clicked(b):

    try:
        # Get the values from the widgets
        id = id_widget.value
        departure_faa = departure_faa_widget.value
        arrive_faa = arrive_faa_widget.value
        carrier = carrier_widget.value
        tailnum = tailnum_widget.value
        sched_dep_time = sched_dep_time_widget.value

        # retrive the correct departure_airport name
        if departure_faa == 'JFK':
            departure_airport = 'John F. Kennedy International Airport'
        elif departure_faa == 'LGA':
            departure_airport = 'LaGuardia Airport'
        else:
            departure_airport = 'Newark Liberty International Airport'

        # Check if new values have been entered
        if arrive_faa == 'New record':
            # if we want new values for the airport then read the values passed throught the optional widgets
            arr_faa = new_arrive_faa_widget.value
            arrive_airport = new_arrive_airport_widget.value
        else:
            # if we have selected a airport that already exists then retrive the airport name associated to the selected faa 
            arr_faa = arrive_faa
            arrive_airport = session.execute(('''SELECT airport FROM arrive WHERE faa = %s ALLOW FILTERING;'''), [arrive_faa])
            arrive_airport = arrive_airport.one().airport
            #print(arrive_airport)

        if carrier == 'New record':
            # if we want new values for the carrier then read the values passed throught the optional widgets
            carrier_value = new_carrier_widget.value
            airline = new_airline_widget.value
        else:
            # if we have selected a airport that already exists then retrive the airport name associated to the selected faa 
            carrier_value = carrier
            airline = session.execute(('''SELECT airline FROM airline_airports WHERE carrier = %s ALLOW FILTERING;'''), [carrier])
            airline = airline.one().airline
            #print(airline)

        if tailnum == 'New record':
            tailnum_value = new_tailnum_widget.value
        else:
            tailnum_value = tailnum
        
        if not sched_dep_time:
                raise ValueError("The scheduled departure time is a mandatory field")

        # Insert the new row into the flights table
        session.execute('''INSERT INTO flights (id, departure_faa, departure_airport, arrive_faa, arrive_airport, carrier, airline, tailnum, sched_dep_time) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)''', 
                            (id, departure_faa, departure_airport, arr_faa, arrive_airport, carrier_value, airline, tailnum_value, sched_dep_time))
        
        # update the 'Airline_airports' table incrementing by 1 the number of flights for the row given by carrier_value & arr_value
        if arrive_faa != 'New_record' and carrier != 'New record':
            current_n_flights = session.execute('''SELECT n_flights FROM airline_airports WHERE arrive_faa = %s AND carrier = %s''', 
                                         (arr_faa, carrier_value)).one().n_flights
            updated_n_flights = int(current_n_flights) + 1 if current_n_flights else 1
            session.execute('''UPDATE airline_airports SET n_flights = %s WHERE arrive_faa = %s AND carrier = %s''', 
                            (updated_n_flights, arr_faa, carrier_value))
        else:
            # if at least one between carrier or arrive_faa is equal to 'New record' then we cannot update the row but we need to insert a new one
            session.execute('''INSERT INTO airline_airports (carrier, faa, n_flights) VALUES (%s, %s, 1)''', 
                            (carrier_value, arr_faa))

        # update the 'Airline_planes' table incrementing by 1 the number of flights for the row given by carrier_value & arr_value
        if tailnum != 'New_record' and carrier != 'New record':
            current_n_flights = session.execute('''SELECT n_flights FROM airline_planes WHERE tailnum = %s AND carrier = %s''', 
                                         (tailnum_value, carrier_value)).one().n_flights
            updated_n_flights = int(current_n_flights) + 1 if current_n_flights else 1
            session.execute('''UPDATE airline_planes SET n_flights = %s WHERE tailnum = %s AND carrier = %s''', 
                            (updated_n_flights, tailnum_value, carrier_value))
        else:
            # if at least one between carrier or tailnum is equal to 'New record' then we cannot update the row but we need to insert a new one
            session.execute('''INSERT INTO airline_planes (carrier, tailnum, n_flights) VALUES (%s, %s, 1)''', 
                            (carrier_value, tailnum_value))

        # update the 'Arrive' table incrementing by 1 the number of flights for the row given by carrier_value & arr_value
        if arrive_faa != 'New record':
            month_value = sched_dep_time.month # this new variable called month_value should be passe to the update statement for retriving the correct row to update
            current_n_flights = session.execute('''SELECT n_flights FROM arrive WHERE faa = %s AND month = %s''', 
                                         (arrive_faa, month_value)).one().n_flights
            updated_n_flights = int(current_n_flights) + 1 if current_n_flights else 1
            session.execute('''UPDATE arrive SET n_flights = %s WHERE faa = %s AND month = %s''', 
                            (updated_n_flights, arr_faa, month_value))
        else:
            # if arrive_faa is equal to 'New record' then we cannot update the row but we need to insert a new one
            month_value = sched_dep_time.month
            session.execute('''INSERT INTO arrive (faa, airport, month, n_flights) VALUES (%s, %s, %s, 1)''', 
                            (arr_faa, arrive_airport, month_value))


        print(f"New flight inserted with ID: {id}")

    except Exception as e:
        print(f"Error: {e}")

# Create widgets for data entry

# Widget for flight ID (disabled as it is auto-generated)
id_widget = widgets.IntText(
    value = generate_unique_id(), description = 'Flight ID:', disabled = True, 
    style = {'description_width': '120px'}, layout = widgets.Layout(border='0')
)

# Dropdown for selecting the departure airport
departure_faa_widget = widgets.Dropdown(
    options = ['JFK', 'LGA', 'EWR'],  # Fixed options for the departing airports 
    description = 'Departure FAA:', style = {'description_width': '150px'}, 
    layout = widgets.Layout(border = '0')
)

# Dropdown for selecting the arrival airport
arrive_faa_widget = widgets.Dropdown(
    options = ['New record'] + distinct_airports,  # Options including a new record option
    description = 'Arrival FAA:', style = {'description_width': '150px'}, 
    layout = widgets.Layout(border = '0')
)

# Dropdown for selecting the carrier
carrier_widget = widgets.Dropdown(
    options = ['New record'] + distinct_carriers,  # Options including a new record option
    description = 'Carrier:', style = {'description_width': '150px'}, 
    layout = widgets.Layout(border = '0')
)

# Dropdown for selecting the tail number
tailnum_widget = widgets.Dropdown(
    options = ['New record'] + distinct_tailnums,  # Options including a new record option
    description = 'Tailnumber:', style = {'description_width': '150px'}, 
    layout = widgets.Layout(border = '0')
)

# Datetime picker for selecting the scheduled departure time
sched_dep_time_widget = widgets.DatetimePicker(
    description = 'Scheduled Departure Time:', style = {'description_width': '200px'}, 
    layout = widgets.Layout(border = '0')
)

# Optional widgets for entering new values
new_arrive_faa_widget = widgets.Text(
    description = 'New Arrival FAA Code:', disabled = False, 
    style = {'description_width': '150px'}, layout=widgets.Layout(border = '0')
)
new_arrive_airport_widget = widgets.Text(
    description = 'Arrival Airport name:', disabled = False, 
    style = {'description_width': '150px'}, layout = widgets.Layout(border = '0')
)
new_carrier_widget = widgets.Text(
    description = 'New Carrier Code:', disabled = False, 
    style = {'description_width': '150px'}, layout = widgets.Layout(border = '0')
)
new_airline_widget = widgets.Text(
    description = 'Airline:', disabled = False, 
    style = {'description_width': '150px'}, layout = widgets.Layout(border = '0')
)
new_tailnum_widget = widgets.Text(
    description = 'New Tailnumber:', disabled = False, 
    style = {'description_width': '150px'}, layout = widgets.Layout(border = '0')
)

# Submit button widget
submit_button = widgets.Button(description = 'Submit')
submit_button.on_click(on_submit_clicked)  # Attach the callback function to the button

# Organize widgets in vertical boxes with indentation for new values
departure_faa_box = widgets.VBox([departure_faa_widget])
arrive_faa_box = widgets.VBox([arrive_faa_widget, widgets.VBox([new_arrive_faa_widget, new_arrive_airport_widget], layout={'padding': '0 0 0 50px'})])
carrier_box = widgets.VBox([carrier_widget, widgets.VBox([new_carrier_widget, new_airline_widget], layout={'padding': '0 0 0 50px'})])
tailnum_box = widgets.VBox([tailnum_widget, widgets.VBox([new_tailnum_widget], layout={'padding': '0 0 0 50px'})])

# Horizontal line for section separation
line = widgets.HTML(value="<hr style='border: 1px solid black;'>")

# Display all widgets in order
display(id_widget, line, departure_faa_box, line, arrive_faa_box, line, carrier_box, line, tailnum_box, line, sched_dep_time_widget, submit_button)

# Callback function to handle the selection of new records
def on_dropdown_change(change):
    # Enable or disable the optional widgets for the new entries based on the dropdown selection
    if change['new'] == 'New record':
        if change['owner'] == arrive_faa_widget:
            new_arrive_faa_widget.disabled = False
            new_arrive_airport_widget.disabled = False
        elif change['owner'] == carrier_widget:
            new_carrier_widget.disabled = False
            new_airline_widget.disabled = False
        elif change['owner'] == tailnum_widget:
            new_tailnum_widget.disabled = False
    else:
        if change['owner'] == arrive_faa_widget:
            new_arrive_faa_widget.disabled = True
            new_arrive_airport_widget.disabled = True
        elif change['owner'] == carrier_widget:
            new_carrier_widget.disabled = True
            new_airline_widget.disabled = True
        elif change['owner'] == tailnum_widget:
            new_tailnum_widget.disabled = True

# Attach the callback function to dropdowns
arrive_faa_widget.observe(on_dropdown_change, names='value')
carrier_widget.observe(on_dropdown_change, names='value')
tailnum_widget.observe(on_dropdown_change, names='value')


IntText(value=334265, description='Flight ID:', disabled=True, layout=Layout(border_bottom='0', border_left='0…

HTML(value="<hr style='border: 1px solid black;'>")

HTML(value="<hr style='border: 1px solid black;'>")

HTML(value="<hr style='border: 1px solid black;'>")

HTML(value="<hr style='border: 1px solid black;'>")

HTML(value="<hr style='border: 1px solid black;'>")

DatetimePicker(value=None, description='Scheduled Departure Time:', layout=Layout(border_bottom='0', border_le…

Button(description='Submit', style=ButtonStyle())

This code can be used to verify the insertion of the new row into the `Flight` table.

In [ ]:
def verify_new_row(id):
    query = session.execute('''SELECT * FROM flights WHERE id = %s''', [id])
    new_row = query.one()
    if new_row:
        print("New row inserted successfully:")
        print(new_row)
    else:
        print("Failed to insert the new row")

# Verify the insertion of the new row by ID
verify_new_row(id_widget.value)

If we want to check if the columns `n_flights` are correcly updated if we add a new flight with an already existing values for some parameters like `carrier`, `arrive_faa` ecc... We can check this running the following example code before and after the update of the row that has the following parameters: 

- `faa = 'BWI` 
- `month = 1`

In [83]:
updated_data = session.execute('''SELECT * FROM arrive WHERE faa = %s AND month = %s''', ('BWI', 1))

for row in updated_data:
    print(row)

Row(faa='BWI', month=1, airport='Baltimore Washington Intl', avg_delay=16.679868698120117, blacklist=['Southwest Airlines Co.', ' Envoy Air', ' ExpressJet Airlines Inc.', ' Endeavor Air Inc.'], n_flights=307, n_rainy_flights=19)
